In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Average, Input
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

# 📁 Paths
BASE_DIR = "C:\\Users\\MANJU\\Desktop\\FYP_Moredata\\split_data"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")

# 🔢 Parameters
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 64
EPOCHS_PHASE1 = 20
EPOCHS_PHASE2 = 10

# 🧪 Preprocessing (choose one common function)
preprocess_func = efficientnet_preprocess  # works well for both models

# 🧪 Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_func,
    rotation_range=40, shear_range=0.2, zoom_range=0.2,
    brightness_range=(0.5, 1.5), horizontal_flip=True, fill_mode='nearest')

val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_func)

# 📦 Data Loaders
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE, class_mode='binary')

val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE, class_mode='binary')

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE, class_mode='binary', shuffle=False)

# ⚖️ Class Weights
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_generator.classes),
                                     y=train_generator.classes)
class_weights = dict(enumerate(class_weights))

# 🧠 Ensemble Model Definition
input_tensor = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# ResNet50 branch
resnet_base = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)
resnet_base.trainable = False
x1 = resnet_base.output
x1 = GlobalAveragePooling2D()(x1)
x1 = Dense(128, activation='relu')(x1)
x1 = Dropout(0.5)(x1)
x1 = Dense(64, activation='relu')(x1)
x1 = Dropout(0.3)(x1)
resnet_out = Dense(1, activation='sigmoid')(x1)

# EfficientNetB0 branch (same input)
efficient_base = EfficientNetB0(weights='imagenet', include_top=False, input_tensor=input_tensor)
efficient_base.trainable = False
x2 = efficient_base.output
x2 = GlobalAveragePooling2D()(x2)
x2 = Dense(128, activation='relu')(x2)
x2 = Dropout(0.5)(x2)
x2 = Dense(64, activation='relu')(x2)
x2 = Dropout(0.3)(x2)
efficient_out = Dense(1, activation='sigmoid')(x2)

# 🧠 Final Output (Averaged prediction)
avg_output = Average()([resnet_out, efficient_out])

# ✅ Define Full Model
model = Model(inputs=input_tensor, outputs=avg_output)

# ⚙️ Compile Model
model.compile(optimizer=Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=['accuracy'])

# 📌 Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# 🚀 Phase 1: Train custom head
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=EPOCHS_PHASE1,
          validation_data=val_generator,
          validation_steps=len(val_generator),
          class_weight=class_weights,
          callbacks=[early_stop, lr_scheduler])

# 🔓 Phase 2: Fine-tuning both branches
resnet_base.trainable = True
efficient_base.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=EPOCHS_PHASE2,
          validation_data=val_generator,
          validation_steps=len(val_generator),
          class_weight=class_weights,
          callbacks=[early_stop, lr_scheduler])

# 🧾 Final Evaluation
loss, accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"\n✅ Final Test Accuracy of Ensemble Model (ResNet50 + EfficientNetB0): {accuracy * 100:.2f}%")


Found 226 images belonging to 2 classes.
Found 48 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


C:\Users\MANJU\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 66s 10s/step - accuracy: 0.6476 - loss: 0.7774 - val_accuracy: 0.5833 - val_loss: 0.6867 - learning_rate: 3.0000e-04
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 29s 7s/step - accuracy: 0.6613 - loss: 0.6191 - val_accuracy: 0.6250 - val_loss: 0.6966 - learning_rate: 3.0000e-04
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 38s 9s/step - accuracy: 0.6492 - loss: 0.5689 - val_accuracy: 0.6667 - val_loss: 0.6718 - learning_rate: 3.0000e-04
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 33s 8s/step - accuracy: 0.7589 - loss: 0.5397 - val_accuracy: 0.6667 - val_loss: 0.6307 - learning_rate: 3.0000e-04
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 31s 8s/step - accuracy: 0.7838 - loss: 0.4877 - val_accuracy: 0.6667 - val_loss: 0.6155 - learning_rate: 3.0000e-04
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 29s 8s/step - accuracy: 0.7835 - loss: 0.4547 - val_accuracy: 0.6875 - val_loss: 0.6659 - learning_rate: 3.0000e-04
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 29s 7s/step - accuracy: 0.8235 - loss: 0.4411 - val